In [2]:
import time
from pinecone import Pinecone, ServerlessSpec

d:\Anaconda\envs\sanka_vir\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pc = Pinecone(
    api_key="pcsk_4AKL3D_8e2Zut6CfAGo6oJ3ZvZydtRd4xHaVvzRYcqHRV7fePJ3AzdXfrXK7iS7L62hQHg"
)

In [18]:
pc.delete_index('my-index')

In [19]:
if 'my-index' not in pc.list_indexes().names():
    pc.create_index(
        name='my-index', 
        dimension=512, 
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )
     
    while not pc.describe_index('my-index').index.status['ready']:
        time.sleep(1)
    
    print("Pinecone Index provisioned")
else:
    print("Pinecone Index Already Provisioned")

index = pc.Index("my-index")

Pinecone Index provisioned


In [24]:
# index.upsert(
#   vectors=[
#     {"id": "A", "values": [0.1, 0.0, 0.0 ]},
#     {"id": "B", "values": [0.0, 0.1, 0.0 ]},
#     {"id": "C", "values": [0.0, 0.0, 0.1 ]},
#     {"id": "D", "values": [0.1, 0.1, 0.1 ]}
#   ]
# )

In [6]:
results = index.fetch(["A", "B", "C", "D"])

print( results )

FetchResponse(namespace='', vectors={'B': Vector(id='B', values=[0.0, 0.1, 0.0], metadata=None, sparse_values=None), 'A': Vector(id='A', values=[0.1, 0.0, 0.0], metadata=None, sparse_values=None), 'C': Vector(id='C', values=[0.0, 0.0, 0.1], metadata=None, sparse_values=None), 'D': Vector(id='D', values=[0.1, 0.1, 0.1], metadata=None, sparse_values=None)}, usage={'read_units': 1})


In [25]:
# # Query the pinecone index passing a query vector
# results2 = index.query(
#     vector=[0.2, 0.0, 0.11],
#     top_k=1,
#     include_values=True
# )

# # Print the results from fetch operation
# print( results2 )

In [26]:
import matplotlib.pyplot as plt
import numpy as np


In [11]:
import open_clip
import torch
from PIL import Image

model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='openai')
model.eval()


d:\Anaconda\envs\sanka_vir\Lib\site-packages\open_clip\factory.py:388: UserWarning: These pretrained weights were trained with QuickGELU activation but the model config does not have that enabled. Consider using a model config with a "-quickgelu" suffix or enable with a flag.
  warnings.warn(


CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-11): 12 x ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((768,), eps=1e-05, elementwise_affine

In [32]:
def image_to_embedding(image_path):
    image = Image.open(image_path).convert("RGB")
    image_input = preprocess(image).unsqueeze(0)

    with torch.no_grad():
        embedding = model.encode_image(image_input).cpu().numpy().flatten()

    return embedding

embedding = image_to_embedding("plot1.png")

index.upsert([#+
    {"id": "plot1", "values": embedding.tolist(), "metadata": {"title": "2008 Birth Rates"}}#+
])#+


{'upserted_count': 1}

In [33]:
query_embedding = image_to_embedding("plot2.png")  # This should return a vector of size 512

# Ensure the query embedding has the correct length
print(len(query_embedding))  # This should print 512


results = index.query(#+
    vector=query_embedding.tolist(),#+
    top_k=3,#+
    include_values=True#+
)

print(results)

512
{'matches': [{'id': 'plot1',
              'score': 0.825685203,
              'values': [0.130080491,
                         -0.165466696,
                         0.118984528,
                         0.353731394,
                         0.315004766,
                         -0.221714526,
                         0.741922617,
                         0.0428458899,
                         0.115382433,
                         -0.189608216,
                         0.46707648,
                         -0.284984022,
                         -0.476096481,
                         -0.273241818,
                         0.375225484,
                         -0.170847937,
                         -0.238401666,
                         -0.000970862806,
                         0.233395517,
                         -0.201985151,
                         0.146761745,
                         -0.0895884186,
                         0.510689378,
                         0.150943696,
    